# **Import Module**

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 642.8 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import zipfile
import shutil
import random
import tensorflow as tf
import torch

from tensorflow.keras.applications import EfficientNetV2L, NASNetLarge, ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import Precision, Recall, AUC
from tensorflow.keras import mixed_precision

BATCH_SIZE = 4
SEED = 1
warnings.filterwarnings('ignore')

# **Import Data and Split Data To Train and Validation**

In [2]:
zip_file_path = '/content/data-model-1.zip'
main_data_dir = '/content/data-model-1'
output_dir = '/content/gambar'
data_dir = '/content/data'

if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    shutil.rmtree(data_dir)
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(data_dir, "train"), exist_ok=True)
    os.makedirs(os.path.join(data_dir, "val"), exist_ok=True)

else:
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(os.path.join(data_dir, "train"), exist_ok=True)
    os.makedirs(os.path.join(data_dir, "val"), exist_ok=True)

if not os.path.exists(main_data_dir):
   with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall("/content")

for folder_name in os.listdir(main_data_dir):
    folder_path = os.path.join(main_data_dir, folder_name)
    new_path = os.path.join(output_dir, folder_name)

    shutil.move(folder_path, new_path)

In [11]:
shutil.rmtree("/content/data/val/.ipynb_checkpoints")

In [5]:
directory = ['/content/gambar/messy', '/content/gambar/tidy']
count = [204 , 226]
pos = 0

for direct in directory:

  for _ in range(count[pos]):
    photo = random.choice(os.listdir(direct))
    old_direct = os.path.join(direct, photo)
    new_direct = f"/content/data/val/messy/{photo}" if pos == 0 else f"/content/data/val/tidy/{photo}"
    shutil.move(old_direct, new_direct)

  pos += 1

In [6]:
directory = ['/content/gambar/messy', '/content/gambar/tidy']
pos = 0

for direct in directory:

  for photo in os.listdir(direct):
    old_direct = os.path.join(direct, photo)
    new_direct = f"/content/data/train/messy/{photo}" if pos == 0 else f"/content/data/train/tidy/{photo}"
    shutil.move(old_direct, new_direct)

  pos += 1

# **Data Augmentation and Create Data Generator**

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [9]:
train_generator = train_datagen.flow_from_directory(
    "/content/data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode= 'binary'
)

Found 1703 images belonging to 2 classes.


In [12]:
val_generator = test_datagen.flow_from_directory(
    "/content/data/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode= 'binary'
)

Found 430 images belonging to 2 classes.


# **Model Initialization**

## **EfficientNet**

In [ ]:
model_ef = EfficientNetV2L(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)

)
model_ef.summary()

473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "efficientnetv2-l"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ rescaling[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_conv      │ (None, 112, 112, 32)   │          9,216 │ stem_activation[0][0]  │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_bn        │ (None, 112, 112, 32)   │            128 │ block1a_project_conv[… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_project_activati… │ (None, 112, 112, 32)   │              0 │ block1a_project_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_add (Add)         │ (None, 112, 112, 32)   │              0 │ block1a_project_activ… │
│                           │                        │                │ stem_activation[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1b_project_conv      │ (None, 112, 112, 32)   │          9,216 │ block1a_add[0][0]      │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1b_project_bn        │ (None, 112, 112, 32)   │            128 │ block1b_project_conv[… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1b_project_activati… │ (None, 112, 112, 32)   │              0 │ block1b_project_bn[0]… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1b_drop (Dropout)    │ (None, 112, 112, 32)   │              0 │ block1b_project_activ… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1b_add (Add)         │ (None, 112, 112, 32)   │              0 │ block1b_drop[0][0],    │
│                           │                        │                │ block1a_add[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1c_project_conv 

 Total params: 117,746,848 (449.17 MB)

 Trainable params: 117,234,272 (447.21 MB)

 Non-trainable params: 512,576 (1.96 MB)

In [ ]:
for layer in model_ef.layers:
  layer.trainable = False

In [ ]:
efficient_model = Sequential([
    model_ef,
    GlobalAveragePooling2D(),
    Flatten(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

## **NASNet**

In [21]:
nasnet_base_model = NASNetLarge(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

nasnet_base_model.trainable = False

343610240/343610240 [==============================] - 9s 0us/step


In [22]:
nasnet_model = Sequential([
    nasnet_base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

## **Resnet**

In [13]:
resnetBase_model = ResNet50(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)
)

resnetBase_model.trainable = False

94765736/94765736 [==============================] - 0s 0us/step


In [14]:
resnet_model = Sequential([resnetBase_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1024, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')]
)

## **Faster RNN**

In [ ]:
cuDNN_model = Sequential([
    LSTM(128, input_shape=(224, 224, 3), return_sequences=True),
    LSTM(64),
    Dense(1, activation="sigmoid")
])

In [ ]:
GRU_model = Sequential([
    GRU(128, input_shape=(224, 224, 3), return_sequences=True),
    GRU(64),
    Dense(1, activation="sigmoid")
])

# **Training**

## **GPU**

In [15]:
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPUs detected.")

GPU 0: Tesla T4


## **Callbacks**

In [20]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_and(self, epoch, logs={}):
    if logs.get('val_auc') > 0.90 and logs.get('val_accuracy') > 0.85:
       print('\n Val Akurasi telah mencapai target')
       self.model.stop_training = True

early_stopping = MyCallback()

reduce_lr = ReduceLROnPlateau(
    monitor='val_auc',
    factor=0.2,
    patience=5,
    min_lr=1e-6
)

checkpoint = ModelCheckpoint(
    '/content/best_model_Nas.keras',
    monitor='val_auc',
    verbose=1,
    save_best_only=True,
    mode='max'
)

In [17]:
metrics = ['accuracy', Precision(), Recall(), AUC(name="auc")]

## **Efficientnet**

In [ ]:
efficient_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [ ]:
history_efficient = efficient_model.fit(
      train_generator,
      epochs=1000,
      validation_data=val_generator,
      callbacks=[early_stopping, reduce_lr, checkpoint]
    )

Epoch 1/1000
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6310 - auc: 0.6773 - loss: 0.6292 - precision_6: 0.6551 - recall_6: 0.7772
Epoch 1: val_auc improved from 0.70568 to 0.78686, saving model to /content/best_model_Efficient.keras
54/54 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.6312 - auc: 0.6773 - loss: 0.6291 - precision_6: 0.6553 - recall_6: 0.7776 - val_accuracy: 0.6884 - val_auc: 0.7869 - val_loss: 0.6135 - val_precision_6: 0.8151 - val_recall_6: 0.5265 - learning_rate: 0.0010
Epoch 2/1000
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 565ms/step - accuracy: 0.6062 - auc: 0.6325 - loss: 0.6434 - precision_6: 0.6491 - recall_6: 0.7888
Epoch 2: val_auc did not improve from 0.78686
54/54 ━━━━━━━━━━━━━━━━━━━━ 41s 670ms/step - accuracy: 0.6063 - auc: 0.6327 - loss: 0.6434 - precision_6: 0.6490 - recall_6: 0.7887 - val_accuracy: 0.7023 - val_auc: 0.7837 - val_loss: 0.6018 - val_precision_6: 0.6899 - val_recall_6: 0.7876 - learning_rate: 0.0010
Epoch 3/1000
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s

## **NasnetLarge**

In [23]:
nasnet_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [24]:
history_NasnetLarge = nasnet_model.fit(
   train_generator,
   epochs=20,
   validation_data=val_generator,
   callbacks=[early_stopping, reduce_lr, checkpoint]
)


Epoch 1/1000
54/54 [==============================] - ETA: 0s - loss: 0.4388 - accuracy: 0.8174 - precision: 0.7908 - recall: 0.8507 - auc: 0.8432
Epoch 1: val_auc improved from -inf to 0.95236, saving model to /content/best_model_Nas.keras
54/54 [==============================] - 92s 1s/step - loss: 0.4388 - accuracy: 0.8174 - precision: 0.7908 - recall: 0.8507 - auc: 0.8432 - val_loss: 0.2704 - val_accuracy: 0.8860 - val_precision: 0.8798 - val_recall: 0.9071 - val_auc: 0.9524 - lr: 0.0010
Epoch 2/1000
54/54 [==============================] - ETA: 0s - loss: 0.2967 - accuracy: 0.8814 - precision: 0.8897 - recall: 0.9159 - auc: 0.9405
Epoch 2: val_auc improved from 0.95236 to 0.97155, saving model to /content/best_model_Nas.keras
54/54 [==============================] - 59s 1s/step - loss: 0.2967 - accuracy: 0.8814 - precision: 0.8897 - recall: 0.9159 - auc: 0.9405 - val_loss: 0.2314 - val_accuracy: 0.9186 - val_precision: 0.9569 - val_recall: 0.8850 - val_auc: 0.9716 - lr: 0.0010
Epo

KeyboardInterrupt: 

## **Resnet**

In [18]:
resnet_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

In [19]:
history_resnet = resnet_model.fit(
    train_generator, epochs=1000,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

Epoch 1/1000
54/54 [==============================] - ETA: 0s - loss: 0.7587 - accuracy: 0.5849 - precision: 0.6269 - recall: 0.7613 - auc: 0.5572
Epoch 1: val_auc improved from -inf to 0.76242, saving model to /content/best_model_Resnet.keras
54/54 [==============================] - 60s 882ms/step - loss: 0.7587 - accuracy: 0.5849 - precision: 0.6269 - recall: 0.7613 - auc: 0.5572 - val_loss: 0.6332 - val_accuracy: 0.6442 - val_precision: 0.6116 - val_recall: 0.8850 - val_auc: 0.7624 - lr: 0.0010
Epoch 2/1000
54/54 [==============================] - ETA: 0s - loss: 0.6329 - accuracy: 0.6588 - precision: 0.6703 - recall: 0.8493 - auc: 0.6592
Epoch 2: val_auc improved from 0.76242 to 0.77878, saving model to /content/best_model_Resnet.keras
54/54 [==============================] - 45s 832ms/step - loss: 0.6329 - accuracy: 0.6588 - precision: 0.6703 - recall: 0.8493 - auc: 0.6592 - val_loss: 0.5775 - val_accuracy: 0.7093 - val_precision: 0.7095 - val_recall: 0.7566 - val_auc: 0.7788 - lr

KeyboardInterrupt: 

## **Faster RNN**

In [ ]:
cuDNN_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

history_CuDNN = cuDNN_model.fit(
    train_generator, epochs=1000,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

In [ ]:
GRU_model.compile(loss=BinaryCrossentropy(),
                 optimizer=Adam(learning_rate=0.001),
                 metrics=metrics)

history_GRU = GRU_model.fit(
    train_generator, epochs=1000,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint]
)